# SETUP PROJECT 1

Importeer pandas voor DataFrames en pyodbc voor database-connecties:

In [36]:
import sqlite3
import pandas as pd
import pyodbc


aenc_conn = sqlite3.connect("../../../Data/Raw/Sqlite/aenc.sqlite")
adventureworks_conn = sqlite3.connect("../../../Data/Raw/Sqlite/AdventureWorks.sqlite")

Database Connectie:

In [37]:
DB_SDM = {"servername": r"143.177.235.181", "database": "SDM_DEDS_P1", "username": "sa", "password": "Str0ngP@ssword"}
DB_NorthWind = {"servername": r"143.177.235.181", "database": "NorthWind", "username": "sa", "password": "Str0ngP@ssword"}
export_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_SDM['servername']};"
    f"DATABASE={DB_SDM['database']};"
    f"UID={DB_SDM['username']};"
    f"PWD={DB_SDM['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

NorthWind_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_NorthWind['servername']};"
    f"DATABASE={DB_NorthWind['database']};"
    f"UID={DB_NorthWind['username']};"
    f"PWD={DB_NorthWind['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

DataFrames maken voor tables:

In [38]:
def create_dataframes_sql(connection, db_type):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    if db_type == "sqlite":
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif db_type == "ssms":
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"

    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        try:
            dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
        except Exception as e:
            print(f"Error reading {table}: {e}")
            continue

    return dictionary


aenc = create_dataframes_sql(aenc_conn, "sqlite")
adventureworks = create_dataframes_sql(adventureworks_conn, "sqlite")
northwind = create_dataframes_sql(NorthWind_conn, "ssms")

#Errors en warnings zijn niet erg hier als het totaal 1 error en 4 UserWarnings zijn.

Error reading sysdiagrams: Could not decode to UTF-8 column 'definition' with text '������'


C:\Users\saadz\AppData\Local\Temp\ipykernel_7680\1853655394.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_7680\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_7680\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:\Us

Dictionaries met DataFrames mergen naar één Dictionary:

In [39]:
sdm = aenc | adventureworks | northwind #Alle drie mergen in één Dictionary met alle DataFrames

sdm['Department'] = sdm['Department'].rename(columns={"dept_id": "DepartmentID", "dept_name": "DepartmentName"})
sdm['HumanResources_Department'] = sdm["HumanResources_Department"].rename(columns={"Name": "DepartmentName"})
sdm['Department'] = pd.concat([sdm["Department"], sdm["HumanResources_Department"]], ignore_index=True)
del sdm['HumanResources_Department']

sdm['Employee'] = sdm['Employee'].rename(columns={'emp_id': 'EmployeeID', 
                                                  'emp_fname': 'FirstName', 
                                                  'emp_lname': 'LastName', 
                                                  'city': 'City', 
                                                  'state': 'Region', 
                                                  'zip_code': 'PostalCode', 
                                                  'phone': 'HomePhone', 
                                                  'start_date': 'HireDate', 
                                                  'birth_date': 'BirthDate'})
sdm['Employees'] = pd.concat([sdm['Employees'], sdm['Employee']], ignore_index=True)
sdm['Employees']
del sdm['Employee']

sdm['Product'] = sdm['Product'].rename(columns={'id': 'ProductID'})
sdm['Employees'] = sdm['Employees'].rename(columns={'dept_id': 'department_id'})

sdm['Category'] = sdm.pop('Categories')
sdm['Employee'] = sdm.pop('Employees')
sdm['Territory'] = sdm.pop('Territories')
sdm['Employee_Territories'] = sdm.pop('EmployeeTerritories')
sdm['Supplier'] = sdm.pop('Suppliers')
sdm['Shipper'] = sdm.pop('Shippers')
sdm['Order'] = sdm.pop('Orders')
sdm['Order_Detail'] = sdm.pop('Order_Details')
sdm['Sales_SalesOrderDetails'] = sdm.pop('Sales_SalesOrderDetail')

# alles lowercase maken omdat het anders niet overeenkomt met SDM database (erg bedankt Jarno...):

def transform_column_name(col):
    col = re.sub(r'([a-z])([A-Z])', r'\1_\2', col)  
    return col.lower()

def transform_dataframe_columns(df_dict):
    updated_dict = {}
    for table_name, df in df_dict.items():
        df = df.rename(columns=lambda col: transform_column_name(col))
        updated_dict[table_name] = df
    return updated_dict

sdm = transform_dataframe_columns(sdm)

# Namen van attributen alsnog handmatig veranderen (alweer bedankt Jarno...):

sdm['Customer'] = sdm['Customer'].rename(columns={
    'id': 'customer_id',
    'state': 'state'
})

sdm['Bonus'] = sdm['Bonus'].rename(columns={
    'emp_id': 'employee_id',
})

sdm['Sales_Order'] = sdm['Sales_Order'].rename(columns={
    'cust_id': 'customer_id',
})

sdm['Sales_Order_Item'] = sdm['Sales_Order_Item'].rename(columns={
    'prod_id': 'product_id',
})

sdm['HumanResources_Employee'] = sdm['HumanResources_Employee'].rename(columns={
    'national_idnumber': 'national_id_number'
})

sdm['Person_Address'] = sdm['Person_Address'].rename(columns={
    'state_province_id': 'state_province'
})

sdm['Production_BillOfMaterials'] = sdm['Production_BillOfMaterials'].rename(columns={
    'bomlevel': 'bom_level'
})

sdm['Production_Product'] = sdm['Production_Product'].rename(columns={
    'discontinued_date': 'discontinued'
})

sdm['Purchasing_PurchaseOrderDetail'] = sdm['Purchasing_PurchaseOrderDetail'].rename(columns={
    'received_qty': 'revieved_qty'
})

sdm['Sales_SalesOrderDetails'] = sdm['Sales_SalesOrderDetails'].rename(columns={
    'sales_order_detail_id': 'sales_order_details_id'
})

dict_order = [
    'Product',
    'Department',
    'State',
    'Employee',
    'Customer',
    'Bonus',
    'Sales_Order',
    'Sales_Order_Item',
    'Region',
    'Territory',
    'Employee_Territories',
    'Category',
    'Supplier',
    'Products',
    'Customers',
    'CustomerDemographics',
    'CustomerCustomerDemo',
    'Shipper',
    'Order',
    'Order_Detail',
    'HumanResources_Employee',
    'Person_Person',
    'Person_Address',
    'Production_BillOfMaterials',
    'Production_ProductCategory',
    'Production_Product',
    'Purchasing_PurchaseOrderHeader',
    'Purchasing_PurchaseOrderDetail',
    'Purchasing_Vendor',
    'Sales_SalesTerritory',
    'Sales_Store',
    'Sales_Customer',
    'Sales_SalesOrderHeader',
    'Sales_SalesOrderDetails'
]               #Hierin komt dictionary order te staan.
                #Dit moet omdat de volgorde van welke tables worden gestuurd naar database belangrijk is.

sdm = {k: sdm[k] for k in dict_order if k in sdm} #Verandert de volgorde van sdm-Dictionary.

SDM in SSMS vullen:

In [40]:
export_cursor = export_conn.cursor()


def format_value(value, column_name, table_name):
    if pd.isna(value):
        return "NULL"
    
    # Special case for boolean values (e.g., discontinued column)
    if isinstance(value, bool):  # Check if the value is boolean
        return "1" if value else "0"  # True -> 1, False -> 0
    
    # Special case: For home_phone and postal_code, treat them as text and wrap them in quotes
    if column_name.lower() in ['home_phone', 'postal_code']:
        return f"'{str(value)}'"
    
    elif isinstance(value, str):
        # Escape single quotes within string values
        return f"'{value.replace("'", "''")}'"
    
    elif isinstance(value, pd.Timestamp):
        # If the value is a pandas Timestamp (date or datetime), format it as a string 'YYYY-MM-DD'
        return f"'{value.strftime('%Y-%m-%d')}'"
    
    elif isinstance(value, bytes):
        # If the value is binary (e.g., image data), convert it to hex format for SQL Server
        hex_value = value.hex()  # Convert binary data to a hex string
        return f"0x{hex_value}"  # Prefix with '0x' for SQL Server to understand it as binary
    
    elif isinstance(value, (int, float)):
        # If value is numeric, return as a string (no quotes for numbers)
        return str(value)
    
    # Default case: if it's any other value, convert it to string and wrap it in quotes
    return f"'{str(value)}'"

# Modify the query generation to wrap table names with square brackets if necessary
def format_table_name(table_name):
    # List of reserved keywords (including 'order')
    reserved_keywords = ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into']
    
    if table_name.lower() in reserved_keywords:
        return f"[{table_name}]"
    
    return table_name

# In your main loop, modify table names
for table_name, df in sdm.items():
    try:
        print(f"Loading table {table_name}:", end=" ")
        
        for index, row in df.iterrows():
            columns = df.columns.tolist()

            values = []
            for col in columns:
                value = row[col]
                # Format the value using the helper function and passing the column name
                values.append(format_value(value, col, table_name))

            column_names = ", ".join(columns)
            value_string = ", ".join(values)
            formatted_table_name = format_table_name(table_name)  # Format table name

            query = f"INSERT INTO {formatted_table_name} ({column_names}) VALUES ({value_string})"

            export_cursor.execute(query)

        export_conn.commit()
        print(f"✓")
    except pyodbc.Error as e:
        print("✗ ERROR")
        print(f"Query: {query}")
        print(f"Error message: {e}")
        print("-" * 80)

export_cursor.close()


Loading table Product: ✓
Loading table Department: ✓
Loading table State: ✓
Loading table Employee: ✓
Loading table Customer: ✓
Loading table Bonus: ✓
Loading table Sales_Order: ✓
Loading table Sales_Order_Item: ✓
Loading table Region: ✓
Loading table Territory: ✓
Loading table Employee_Territories: ✓
Loading table Category: ✓
Loading table Supplier: ✓
Loading table Products: ✓
Loading table Customers: ✓
Loading table CustomerDemographics: ✓
Loading table CustomerCustomerDemo: ✓
Loading table Shipper: ✓
Loading table Order: ✓
Loading table Order_Detail: ✓
Loading table HumanResources_Employee: ✗ ERROR
Query: INSERT INTO HumanResources_Employee (business_entity_id, national_id_number, login_id, organization_node, organization_level, job_title, birth_date, marital_status, gender, hire_date, salaried_flag, vacation_hours, sick_leave_hours, current_flag, rowguid, modified_date, department_id) VALUES ('1', '295847284', 'adventure-works\ken0', NULL, '', 'Chief Executive Officer', '1979-01-29